## Import

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install scikit-learn
# !pip install tqdm
# !pip install seaborn
!pip install prophet
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 9.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.7/759.7 KB 9.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 9.2 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.4 MB/s eta 0:00:0000:0100:01m
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732017 sha256=a424e2cb4e27e80fe2169862adb017adff66362e9f2f8fcf84bf90a73ad901a6
  Stored in directory: /home/do/.cache/pip/wheels/d6/67/78/aa2e8d108639dd23a5e9e72a4fc88bb44f5541894382712f48
Successfully built pymeeus
Defaulting to user installation because normal site-packages is not writ

In [2]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 289.2 kB/s eta 0:00:0000:0100:02


In [4]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
import torch

from prophet import Prophet
from datetime import datetime, timedelta, timezone
from itertools import combinations, product

import gc
import warnings
warnings.filterwarnings(action='ignore') 

# Device & Path

In [5]:
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
torch.cuda.is_available()

True

In [12]:
# DIR
DATA_DIR  = '../DATA'
# TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TRAIN_CSV  = os.path.join(DATA_DIR, 'train.csv')
SAMPLE_DIR = os.path.join(DATA_DIR, 'example_test_files', 'sample_submission.csv')

## Hyperparameter Setting

In [13]:
CFG = {
    # 'EPOCHS':1,
    # 'LEARNING_RATE':3e-4,
    'SEED':42
}

## Fixed RandomSeed

In [14]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True


seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [15]:
df_train = pd.read_csv(TRAIN_CSV)
df_ss = pd.read_csv(SAMPLE_DIR)
df_train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [17]:
print(df_train.columns)
print(df_train.shape)
print(df_train.info())

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')
(5237980, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237980 entries, 0 to 5237979
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap     

## columns 


df_train

In [145]:
# change columns name
df = df_train.rename(columns={
    '일자': 'date',
    '종목코드': 'code',
    '종목명': 'name',
    '거래량': 'volume',
    '시가': 'open',
    '고가': 'high',
    '저가': 'low',
    '종가': 'close'
})
# datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df_price = df.pivot(index='date', columns='code', values='close')

df_processed = df.copy()
le = preprocessing.LabelEncoder()
df_processed['code'] = le.fit_transform(df_processed['code'])  
df_processed.drop(columns=['name'], inplace=True)

df_processed.set_index('date', inplace=True)


,date,code,name,volume,open,high,low,close
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050


In [149]:
# min max data
print(f'min of data is {df.date.min()}')
print(f'max of date is {df.date.max()}')
print(f'sum of days are {df.date.max()-df.date.min()}')

df_test = df[df['code']=='A238490'].copy()
# weekend
df_test['is_weekend'] = df_test['date'].apply(lambda x: x.weekday() >= 5)

# 'is_weekend' columns False count 
weekday_count = df_test[~df_test['is_weekend']].shape[0]

print(f"Number of weekdays: {weekday_count}")

min of data is 2021-06-01 00:00:00
max of date is 2023-05-30 00:00:00
sum of days are 728 days 00:00:00
Number of weekdays: 494


In [157]:
# Rolling window size
window_size = 3

# Apply rolling window
df['close_rolling_mean'] = df.groupby('code')['close'].transform(lambda x: x.rolling(window=window_size).mean())

# Drop rows with NaN values
df = df.dropna()

# Display the updated DataFrame
print(df)


             date     code    name   volume   open   high    low  close  \
4000   2021-06-03  A060310      3S   144470   2900   2925   2875   2900   
4001   2021-06-03  A095570  AJ네트웍스    78440   5800   5800   5650   5720   
4002   2021-06-03  A006840   AK홀딩스   115028  36950  38050  36300  36300   
4003   2021-06-03  A054620     APS   204621  14500  14700  14100  14550   
4004   2021-06-03  A265520   AP시스템   176959  29300  29650  29100  29350   
...           ...      ...     ...      ...    ...    ...    ...    ...   
987995 2023-05-30  A189980  흥국에프엔비   272284   3005   3035   2955   2980   
987996 2023-05-30  A000540    흥국화재    50218   3250   3255   3195   3215   
987997 2023-05-30  A003280    흥아해운   130664   1344   1395   1340   1370   
987998 2023-05-30  A037440      희림   141932   9170   9260   9170   9200   
987999 2023-05-30  A238490      힘스  2611843   6410   8220   6300   8220   

                   RowId  close_rolling_mean  
4000    20210603_A060310         2906.666667  
4001 

## Label-Encoding

In [160]:
tscv = TimeSeriesSplit(n_splits=5)

def timeseries_cv(df):
    results_df = pd.DataFrame(columns=['code', 'final_return'])

    for code in df['code'].unique():
        df_company = df[df['code'] == code]

        final_returns = []  # Store all final returns for this company

        for train_index, val_index in tscv.split(df_company):
            X_train, X_val = df_company[['volume', 'open', 'high', 'low', 'close_rolling_mean']].iloc[train_index], df_company[['volume', 'open', 'high', 'low', 'close_rolling_mean']].iloc[val_index]
            y_train, y_val = df_company['close'].iloc[train_index], df_company['close'].iloc[val_index]

            model = LGBMRegressor()
            model.fit(X_train, y_train)

            last_row = X_val.iloc[-1][['volume', 'open', 'high', 'low', 'close_rolling_mean']].copy()

            future_returns = []
            for i in range(15):
                future_date = last_row.name + timedelta(days=i+1)
                prediction = model.predict(last_row.values.reshape(1,-1))
                future_returns.append(prediction[0])

                # Update the last row with the predicted value
                last_row['close_rolling_mean'] = prediction[0]

            if future_returns[0] != 0:
                final_return = (future_returns[-1] - future_returns[0]) / future_returns[0]
            else:
                final_return = 0

            final_returns.append(final_return)

        # Compute the average final return for this company
        avg_final_return = sum(final_returns) / len(final_returns)

        results_df = results_df.append({'code': code, 'final_return': avg_final_return}, ignore_index=True)

    return results_df


In [161]:
# df_processed.drop(columns=['index'], inplace=True)
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 984000 entries, 2021-06-03 to 2023-05-30
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   code                984000 non-null  int64  
 1   volume              984000 non-null  int64  
 2   open                984000 non-null  int64  
 3   high                984000 non-null  int64  
 4   low                 984000 non-null  int64  
 5   close               984000 non-null  int64  
 6   close_rolling_mean  984000 non-null  float64
dtypes: float64(1), int64(6)
memory usage: 60.1 MB


## !!RUN!!

In [116]:
results_df = timeseries_cv(df_processed)
results_df

,code,final_return
0,971.0,0.005571
1,1267.0,0.000000
2,280.0,0.001052
3,929.0,0.000000
4,1789.0,0.000000
...,...,...
1995,1562.0,0.000176
1996,27.0,0.000000
1997,146.0,0.001400
1998,732.0,-0.000952


# Record path

In [240]:
# 시간 고유값 
kst = timezone(timedelta(hours=9))        
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")

# 기록 경로
RECORDER_DIR = os.path.join(DATA_PATH, 'results', train_serial)

# 현재 시간 기준 폴더 생성
os.makedirs(RECORDER_DIR, exist_ok=True)    
RESULT_PATH = os.path.join(RECORDER_DIR, 'submission.csv')

## Submission

In [125]:
results_df['rank'] = results_df['final_return'].rank(method='first', ascending=False).astype('int')
results_df['code']= le.inverse_transform(results_df['code'].astype(int))
results_df.sort_values(by=['rank'], inplace=True)
results_df = results_df[['code', 'rank']]
results_df.rename(columns={'code': '종목코드', 'rank': '순위'}, inplace=True)
results_df.to_csv(RESULT_PATH, index=False)
results_df

,종목코드,순위
987,A183190,1
1064,A097780,2
1339,A056090,3
928,A005800,4
1535,A016790,5
...,...,...
1873,A226440,1996
296,A035460,1997
1590,A009730,1998
1360,A093230,1999


## Diff calculation

        code  final_return_mean
0      971.0          -0.001184
1     1267.0          -0.033504
2      280.0           0.021727
3      929.0          -0.011741
4     1789.0           0.010884
...      ...                ...
1995  1562.0           0.004717
1996    27.0           0.001612
1997   146.0          -0.017114
1998   732.0          -0.003577
1999  1715.0           0.003874

[2000 rows x 2 columns]


,code,final_return_mean,rank
311,A293580,0.174600,1
1127,A086520,0.097408,2
1036,A096610,0.092526,3
1510,A270520,0.090964,4
212,A002710,0.088835,5
...,...,...,...
1980,A205470,-0.383986,1996
1481,A000480,-0.467699,1997
360,A214870,-0.650305,1998
912,A016590,-0.666215,1999


In [213]:
# 각 코드별 최종 수익률 계산
final_returns = {}
for code, df in df_predictions.items():
    final_returns[code] = (df['trend'].iloc[-1] - df['trend'].iloc[0]) / df['trend'].iloc[0]

# 평균 최종 수익률 계산
average_final_returns = {}
for code, final_return in final_returns.items():
    average_final_returns[code] = final_return.mean()

# 평균 최종 수익률에 따라 코드 순위 매기기
ranked_codes = sorted(average_final_returns, key=average_final_returns.get, reverse=True)
ranks = {code: rank for rank, code in enumerate(ranked_codes, 1)}

# 각 분리된 데이터프레임에 순위 열 추가
for code, df in df_predictions.items():
    df['rank'] = ranks[code]

# 모든 분리된 데이터프레임 연결
df_predictions = pd.concat(df_predictions.values())
# 코드, 평균 최종 수익률, 순위(rank)를 포함하는 데이터프레임 생성
df_final = pd.DataFrame({
    'code': ranked_codes,
    'final_return': [average_final_returns[code] for code in ranked_codes],
    'rank': [ranks[code] for code in ranked_codes]
})


df_final['code']= le.inverse_transform(df_final['code'].astype(int))
df_final = df_final[['code', 'rank']]
df_final.rename(columns={'code': '종목코드', 'rank': '순위'}, inplace=True)
df_final.to_csv(RESULT_PATH, index=False)
df_final


In [215]:
df_final = df_final[['code', 'rank']]
df_final.rename(columns={'code': '종목코드', 'rank': '순위'}, inplace=True)
df_final.to_csv(RESULT_PATH, index=False)
df_final

,종목코드,순위
0,A101730,1
1,A010580,2
2,A101670,3
3,A145210,4
4,A073570,5
...,...,...
1995,A054180,1996
1996,A263540,1997
1997,A214270,1998
1998,A003280,1999


In [127]:
def prep_prices(df):
    
    from decimal import ROUND_HALF_UP, Decimal
    
    pcols = ['open', 'high', 'low', 'close']

    #df.ExpectedDividend.fillna(0,inplace=True) # No column 'ExpectedDividend' in the given data
    
    def qround(x):
        return float(Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP))
    
    def adjust_prices(df_code):
        df_code = df_code.sort_values("date", ascending=False)
        #df_code.loc[:, "CumAdjust"] = df_code["AdjustmentFactor"].cumprod() # No column 'AdjustmentFactor' in the given data

        # generate adjusted prices
        for p in pcols:     
            df_code.loc[:, p] = df_code[p].apply(qround)
        df_code.loc[:, "volume"] = df_code["volume"]
        df_code.ffill(inplace=True)
        df_code.bfill(inplace=True)
        
        # generate and fill Targets
        #df_code.loc[:, "Target"] = df_code.Close.pct_change().shift(-2).fillna(df_code.Target).fillna(0) # No column 'Target' in the given data
        #df_code.Target.fillna(0,inplace=True)

        return df_code

    # generate Adjusted
    df = df.sort_values(["code", "date"])
    df = df.groupby("code").apply(adjust_prices).reset_index(drop=True)
    df = df.sort_values("date")
    return df


In [128]:
df_processed

,code,volume,open,high,low,close,close_rolling_mean
date,,,,,,,
2021-06-03,971,144470,2900,2925,2875,2900,2906.666667
2021-06-03,1267,78440,5800,5800,5650,5720,5766.666667
2021-06-03,280,115028,36950,38050,36300,36300,35716.666667
2021-06-03,929,204621,14500,14700,14100,14550,14666.666667
2021-06-03,1789,176959,29300,29650,29100,29350,29183.333333
...,...,...,...,...,...,...,...
2023-05-30,1562,272284,3005,3035,2955,2980,3021.666667
2023-05-30,27,50218,3250,3255,3195,3215,3248.333333
2023-05-30,146,130664,1344,1395,1340,1370,1357.000000


- test를 위한 코드

In [163]:
df_processed.reset_index(inplace=True)
# Process the data using the given function
df_train = prep_prices(df_processed)

# Select data from 2021-06-03 to 2023-05-30
mask = (df_train['date'] >= "2021-06-03") & (df_train['date'] <= "2023-05-30")
df_train = df_train.loc[mask]

# Prepare the data for Prophet
df_train = df_train.rename(columns={"date": "ds", "close": "y"})

# Separate features by 'code'
data_dict = {}
unique_codes = df_train['code'].unique()

for code in unique_codes:
    data_dict[code] = df_train[df_train['code'] == code][['ds', 'y']]


In [168]:
data_dict[661]

,ds,y
325703,2021-06-03,1055.0
325702,2021-06-04,1055.0
325701,2021-06-07,1055.0
325700,2021-06-08,1055.0
325699,2021-06-09,1055.0
...,...,...
325216,2023-05-23,2110.0
325215,2023-05-24,2110.0
325214,2023-05-25,2110.0
325213,2023-05-26,2110.0


In [167]:
model = Prophet()
model.fit(data_dict[661])

# # Create a dataframe to hold the future dates
future = model.make_future_dataframe(periods=1)

# Predict on the future dates
forecast = model.predict(future)
predictions_dict = {}
# Save the predictions to the dictionary
predictions_dict[code] = forecast

13:12:51 - cmdstanpy - INFO - Chain [1] start processing
13:12:51 - cmdstanpy - INFO - Chain [1] done processing
13:12:51 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
13:12:51 - cmdstanpy - INFO - Chain [1] start processing
13:12:51 - cmdstanpy - INFO - Chain [1] done processing
13:12:51 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin random seed=34726 data file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/9vzpwo1g.json init=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/4jmapd4e.json output file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/prophet_model00otapfi/prophet_model-20230708131251.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[56554]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <7F076692-0450-3E56-A346-2F49A3E374A6> /Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/pip-install-2u8ye0j9/prophet_c0ad0c1de14b4676964f42b5b875a676/build/lib.macosx-10.9-universal2-cpython-39/prophet/stan_model/cmdstan-2.31.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such f

- 아마 미래기간에 대한 데이터 프레임을 따로 생성해줘야할 듯
- In sample test 진행 후에 out of sample test를 진행한다
- 밑에 코드는 예시코드 

In [130]:
from prophet import Prophet
# Initialize a dictionary to hold prediction dataframes
predictions_dict = {}

# Loop over each code
for code in unique_codes:
    # Initialize Prophet and fit the data
    model = Prophet()
    model.fit(data_dict[code])

    # Create a dataframe to hold the future dates
    future = model.make_future_dataframe(periods=15)

    # Predict on the future dates
    forecast = model.predict(future)

    # Save the predictions to the dictionary
    predictions_dict[code] = forecast

# Concatenate all prediction dataframes
df_predictions = pd.concat(predictions_dict.values())


22:17:47 - cmdstanpy - INFO - Chain [1] start processing
22:17:48 - cmdstanpy - INFO - Chain [1] done processing
22:17:48 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
22:17:48 - cmdstanpy - INFO - Chain [1] start processing
22:17:48 - cmdstanpy - INFO - Chain [1] done processing
22:17:48 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin random seed=2208 data file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/gcazj4ay.json init=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/r9piiqrl.json output file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/prophet_modelyztio2l0/prophet_model-20230706221748.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[34605]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <7F076692-0450-3E56-A346-2F49A3E374A6> /Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/pip-install-2u8ye0j9/prophet_c0ad0c1de14b4676964f42b5b875a676/build/lib.macosx-10.9-universal2-cpython-39/prophet/stan_model/cmdstan-2.31.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such fi

In [ ]:
# read pkl
df_predictions = pd.read_pickle(os.path.join(DATA_PATH, 'predictions_dict.pkl'))
# df_processed
true_y = df_processed[['code','close']][df_processed['date']=='2023-05-30']

# Initialize the DataFrame
final_df = pd.DataFrame(columns=['code', 'final_return_mean'])

for item in true_y['code']:
    y_true = true_y[true_y['code']==item]['close']
    y_hat = df_predictions[item]['yhat'][df_predictions[item]['ds']=='2023-05-30']
    diff = y_true.values[0] - y_hat.values[0]
    results = df_predictions[item]['yhat']+diff
    results = results[-15:]

    # Calculate final return
    final_return = (results.iloc[-1] - results.iloc[0]) / results.iloc[0]

    # Append to DataFrame
    final_df = final_df.append({'code': item, 'final_return_mean': final_return}, ignore_index=True)

# Print the final DataFrame
print(final_df)

final_df['rank'] = final_df['final_return_mean'].rank(method='first', ascending=False).astype('int')
final_df['code']= le.inverse_transform(final_df['code'].astype(int))
final_df.sort_values(by=['rank'], inplace=True)
final_df = final_df[['code', 'rank']]
final_df.rename(columns={'code': '종목코드', 'rank': '순위'}, inplace=True)
final_df.to_csv(RESULT_PATH, index=False)
final_df


## In-sample Forecast

-   트레인셋과 테스트 셋으로 확인하는 과정




In [177]:
# train set 마지막 15일 
last_1month_in = list()

for i in range(15,31):
    last_1month_in.append(['2023-05-%02d' % i])

last_1month_in = pd.DataFrame(last_1month_in, columns = ['ds'])
last_1month_in['ds']= pd.to_datetime(last_1month_in['ds'])


In [ ]:
m.fit(data_dict[661])

In [180]:
data_dict[661]

,ds,y
325703,2021-06-03,1055.0
325702,2021-06-04,1055.0
325701,2021-06-07,1055.0
325700,2021-06-08,1055.0
325699,2021-06-09,1055.0
...,...,...
325216,2023-05-23,2110.0
325215,2023-05-24,2110.0
325214,2023-05-25,2110.0
325213,2023-05-26,2110.0


In [181]:
tt = data_dict[661]
tt = tt.reset_index(drop=True)
tt

,ds,y
0,2021-06-03,1055.0
1,2021-06-04,1055.0
2,2021-06-07,1055.0
3,2021-06-08,1055.0
4,2021-06-09,1055.0
...,...,...
487,2023-05-23,2110.0
488,2023-05-24,2110.0
489,2023-05-25,2110.0
490,2023-05-26,2110.0


In [184]:
last_1month_in



,ds
0,2023-05-15
1,2023-05-16
2,2023-05-17
3,2023-05-18
4,2023-05-19
5,2023-05-20
6,2023-05-21
7,2023-05-22
8,2023-05-23
9,2023-05-24


In [191]:
!brew install tbb


161045.28s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Running `brew update --auto-update`...
Installing from the API is now the default behaviour!
You can save space and time by running:
  brew untap homebrew/core
==> Downloading https://formulae.brew.sh/api/formula_tap_migrations.jws.json
######################################################################### 100.0%
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
access                                   libversion
adb-enhanced                             libvisual
aftman                                   libvisual-plugins
alass                                    libvisual-projectm
ansible@7                                libzen
apko                                     lowdown
ares                                     lpeg
argparse                                 lr
ast-grep                                 ls-lint
aws-amplify                              mariadb@10.11
bashate                                  melange
bbot                      

In [199]:
!export DYLD_LIBRARY_PATH=/usr/local/lib:$DYLD_LIBRARY_PATH


161247.64s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [200]:
!pip uninstall -y prophet
!pip install prophet

161316.68s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Found existing installation: prophet 1.1.4
Uninstalling prophet-1.1.4:
  Successfully uninstalled prophet-1.1.4


161322.35s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
  Using cached prophet-1.1.4-py3-none-macosx_11_0_universal2.whl


In [201]:
from prophet import Prophet

In [202]:
# tt.reset_index(inplace=True)
# example
m = Prophet()
m.fit(tt)
# 에측
forecast = m.predict(last_1month_in)

14:22:43 - cmdstanpy - INFO - Chain [1] start processing
14:22:44 - cmdstanpy - INFO - Chain [1] done processing
14:22:44 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6
Optimization terminated abnormally. Falling back to Newton.
14:22:44 - cmdstanpy - INFO - Chain [1] start processing
14:22:44 - cmdstanpy - INFO - Chain [1] done processing
14:22:44 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 6 Unknown error: -6


RuntimeError: Error during optimization! Command '/Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin random seed=48675 data file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/m_f32k4d.json init=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/3bs2him3.json output file=/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/tmpxzh7k3d3/prophet_modeld5yemfko/prophet_model-20230708142244.csv method=optimize algorithm=newton iter=10000' failed: console log output:

dyld[59171]: Library not loaded: @rpath/libtbb.dylib
  Referenced from: <7F076692-0450-3E56-A346-2F49A3E374A6> /Users/admin/Library/Python/3.9/lib/python/site-packages/prophet/stan_model/prophet_model.bin
  Reason: tried: '/private/var/folders/nh/_4ldy_ss7yv872vc0r53p2jw0000gn/T/pip-install-2u8ye0j9/prophet_c0ad0c1de14b4676964f42b5b875a676/build/lib.macosx-10.9-universal2-cpython-39/prophet/stan_model/cmdstan-2.31.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such f

## 메모리 확인

In [187]:
!pip install psutil


160674.60s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


In [189]:
import psutil

# Get the memory details
memory_info = psutil.virtual_memory()

print(f"Total memory: {memory_info.total / (1024.0 ** 3)} GB")
print(f"Available memory: {memory_info.available / (1024.0 ** 3)} GB")
print(f"Used memory: {memory_info.used / (1024.0 ** 3)} GB")
print(f"Memory percent used: {memory_info.percent}%")

Total memory: 8.0 GB
Available memory: 1.6962738037109375 GB
Used memory: 2.98797607421875 GB
Memory percent used: 78.8%


In [245]:
!pip install finance-datareader

176423.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable


In [246]:
!pip install pykrx


176434.32s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 4.8 MB/s eta 0:00:00a 0:00:01


#### 앞으로 할것 
- validation data 구축 -> 비교 분석
- shape ratio 공식
- prophet / lightgbm / transformer 비교